In [ ]:
# miningPoolMoniter

import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import traceback
import os

# Credentials
walletAddress=''

# Path to chrome driver
browser = webdriver.Chrome('.\\chromedriver.exe')

# Output file
file_name = 'poolData.csv'

# Navigate to mining pools and enter wallet address

print('Navigating to site')
browser.get('https://supportxmr.com/')
time.sleep(5)
wallet = browser.find_element_by_css_selector('#AddrField')
wallet.send_keys(walletAddress)
time.sleep(1)
wallet.send_keys(Keys.ENTER)
time.sleep(5)

def dataGrab():
    """
    Function extracts required information from webpage one every 60 seconds,
    outputing to file once every hour.
    """
    
    # initialise data frame to store data
    df = pd.DataFrame(columns=['Worker Count', 'Hashes', 
                           'Shares','Pending','Paid',
                           'Network Rate','Pool Rate',
                           'Time stamp','Hash Rate'
                          ])
    # Loop control variables
    run = True ; counter = 0
    
    while run:
        counter+=1
        
        # Retrive HTML
        html=browser.page_source
        soup = BeautifulSoup(html, 'html.parser')
        print('Extracting Data')
        try:
            # Extract required information from HTML doc
            workerCount = soup.find_all("div",{'id':'MinerWorkerCount'})
            totalHashes = soup.find_all("div",{'id':'MinerHashes'})
            minerShares = soup.find_all("div",{'id':'MinerShares'})
            pendingXMR = soup.find_all("span",{'id':'DashPending'})
            paidXMR = soup.find_all("span",{'id':'DashPaid'})
            networkRate, poolRate = soup.find_all("text",{'class':'txtmed C1fl'})
            rates = soup.find_all("circle",{'class':'ToolTip C1fl_hov'})
            
            # Remove HTML syntax and format data
            var=str(rates[-1])
            var2=var.split()
            hashRateString=var2[5] ; timeStampString = var2[6]
            var3 = hashRateString.split('=') ; var4 = timeStampString.split('=')
            hashRate = var3[-1] ; timeStamp = var4[-1]
            hashRate=float(hashRate.strip("\""))
            timeStamp=float(timeStamp.strip("\""))
            
            # Add extracted data to data frame
            newEntry = {'Worker Count': workerCount[0].text,
                     'Hashes':totalHashes[0].text,
                     'Shares':minerShares[0].text,
                     'Pending':pendingXMR[0].text,
                     'Paid':paidXMR[0].text,
                     'Network Rate':networkRate.text,
                     'Pool Rate':poolRate.text,
                     'Time stamp':timeStamp,
                     'Hash Rate':hashRate
                   }
            df=df.append(newEntry, ignore_index=True)
            
            # Output new data to screen
            for key in newEntry:
                print(key,' = ',newEntry[key])
            print('\n')
            

            
        # Output any errors to screen
        except Exception:
            traceback.print_exc()
            
        def writeData(df):
            """
            Write dataframe to file, creates newfile with headers if file dosen't exist,
            appends and omits header if it does
            """
            # if file does not exist write header 
            if not os.path.isfile(file_name):
                df.to_csv(file_name, header='column_names',index=False)
            
            # else it exists so append without writing the header
            else: 
                df.to_csv(file_name, mode='a', header=False,index=False)
                
        # Output to file, delete and reinitialise dataframe to avoid duplicate entrys
        if counter % 5 == 0:
            writeData(df)
            del(df)
            df = pd.DataFrame(columns=['Worker Count', 'Hashes', 
                           'Shares','Pending','Paid',
                           'Network Rate','Pool Rate',
                           'Time stamp','Hash Rate'
                          ])
              
        time.sleep(60)
        browser.refresh()

        
if name is __main__:
    df = dataGrab()

Navigating to site


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=110.0.5481.180)
